# **Titanic Survival Prediction**

> # **PIPELINE**

way to chain multiple data processing steps and a final estimator (model) into a single object. It ensures that all steps are applied in the correct order during both training and prediction.

**What goes INSIDE a Pipeline**
```
Raw Features
   ↓
Preprocessing
   ↓
Feature Engineering
   ↓
Model
```

**What stays OUTSIDE**



1.   Train-test split

2. Cross-validation

3. Evaluation (metrics)

4. Deployment logic



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv('titanic_data.csv')

In [3]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df=df.drop(columns=['PassengerId','Name','Ticket','Cabin'])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(df.drop(['Survived'],axis=1),df['Survived'],test_size=0.2,random_state=42)

In [6]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [7]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [8]:
#impute transformer
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(strategy='median'),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6]),
],remainder= 'passthrough')

In [9]:
#oneHotEncoding
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')


In [10]:
#Scaling
trf3=ColumnTransformer([
    ('pclass_age_sib_par_scaling',MinMaxScaler(),slice(0,10))
])

In [11]:
#feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [12]:
#model
trf5=DecisionTreeClassifier()

In [13]:
pipeline=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# **1️⃣ Pipeline (explicit, named steps)**

```
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])
```
**Characteristics**

✅ You name each step yourself

✅ Very clear & readable

✅ Best for GridSearchCV, debugging, production

⚠️ Slightly more verbose


# **2️⃣ make_pipeline (shortcut / convenience)**
```
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)
```
**Characteristics**

✅ Less code

✅ Auto-generates step names

⚠️ Step names are class-based, not obvious

⚠️ Harder to tune in complex cases

In [14]:
#pipeline=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [15]:
pipeline.fit(X_train,Y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age',
                                                  SimpleImputer(strategy='median'),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('pclass_age_sib_par_scaling',
                                                  MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x78bb9fb00400>)),
                ('trf5', DecisionTreeClassifier())])

In [16]:
pipeline.named_steps['trf1'].transformers_[0][1].statistics_

array([28.])

In [17]:
pipeline.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [18]:
Y_pred=pipeline.predict(X_test)

In [19]:
round(accuracy_score(Y_test,Y_pred)*100,ndigits=2)

62.57

> # **Cross-Validation using Pipeline**





In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipeline,X_train,Y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)

> # **GridSearch using PipeLine**





In [21]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [22]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipeline, params, cv=5, scoring='accuracy')
grid.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(strategy='median'),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('pclass_age_sib_par_scaling',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x78bb9fb00400>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [23]:
grid.best_score_

np.float64(0.6391214419383433)